In [2]:
import sqlite3
import os
import pandas as pd

In [6]:
os.chdir("/Users/annecastaneda/Downloads/dataverse_files")




In [7]:
try:
    os.remove('airline2.db')
except OSError:
    pass



In [8]:
conn = sqlite3.connect('airline2.db')

In [9]:
airports_df = pd.read_csv("airports.csv")
carriers_df = pd.read_csv("carriers.csv")
planes_df = pd.read_csv("plane-data.csv")

In [10]:
airports_df.to_sql('airports', con = conn, index = False)
carriers_df.to_sql('carriers', con = conn, index = False)
planes_df.to_sql('planes', con = conn, index = False)

5029

In [11]:
c = conn.cursor()
c.execute('''
CREATE TABLE ontime (
  Year int,
  Month int,
  DayofMonth int,
  DayOfWeek int,
  DepTime  int,
  CRSDepTime int,
  ArrTime int,
  CRSArrTime int,
  UniqueCarrier varchar(5),
  FlightNum int,
  TailNum varchar(8),
  ActualElapsedTime int,
  CRSElapsedTime int,
  AirTime int,
  ArrDelay int,
  DepDelay int,
  Origin varchar(3),
  Dest varchar(3),
  Distance int,
  TaxiIn int,
  TaxiOut int,
  Cancelled int,
  CancellationCode varchar(1),
  Diverted varchar(1),
  CarrierDelay int,
  WeatherDelay int,
  NASDelay int,
  SecurityDelay int,
  LateAircraftDelay int
)
''')

In [15]:
conn.commit()

In [16]:
for year in range(2000, 2006):
    ontime_temp_df = pd.read_csv(str(year)+".csv")
    ontime_temp_df.to_sql('ontime', con = conn, if_exists = 'append', index = False)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 343: invalid continuation byte

In [17]:
conn.commit()

In [18]:
c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
''')

In [19]:
print(c.fetchone()[0], "has the lowest associated average departure delay.")

737-2Y5 has the lowest associated average departure delay.


In [20]:
c.execute('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC
''')

In [21]:
print(c.fetchone()[0], "has the highest number of inbound flights (excluding cancelled flights)")

Chicago has the highest number of inbound flights (excluding cancelled flights)


In [22]:
c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC
''')

In [23]:
print(c.fetchone()[0],"has the highest number of cancelled flights")

United Air Lines Inc. has the highest number of cancelled flights


In [24]:
c.execute('''
SELECT
q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
(
  SELECT carriers.Description AS carrier, COUNT(*) AS numerator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1 AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q1 JOIN
(
  SELECT carriers.Description AS carrier, COUNT(*) AS denominator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC
''')

In [25]:
print(c.fetchone()[0], "highest number of cancelled flights, relative to their number of total flights")


United Air Lines Inc. highest number of cancelled flights, relative to their number of total flights


In [26]:
conn.close()